In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from xgboost.spark import SparkXGBClassifier
from pyspark.ml import PipelineModel
from pyspark.sql.types import *
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.functions import col
import pandas as pd
from time import time
# from miniapps.utils import nice_time
# from miniapps.utils import create_date_time_str
from pyspark.ml.feature import IndexToString, StringIndexer
import os
import warnings
from datetime import date
from pyspark.ml.classification import GBTClassifier 

In [2]:
from pyspark.ml.classification import GBTClassifier 

In [3]:
CONFIG_SPARK = [
        # ("spark.driver.memory", '4g'),
        # ("spark.driver.cores", '2'),
        ("spark.executor.memory", '5g'),
        ("spark.executor.cores", '3'),
        ("spark.executor.instances", '5')
    ]

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from time import time
import os

os.environ['PYSPARK_PYTHON'] = './environment/bin/python'
s = time()
configs = CONFIG_SPARK
conf = SparkConf().setAll(configs)

spark = SparkSession \
    .builder \
    .appName("crosssla") \
    .config(conf=conf) \
    .config("spark.yarn.dist.archives","hdfs:///virtual_env/congtt3/pyspark_env_python39.tar.gz#environment")\
    .getOrCreate()
print("done to get spark:")
print(spark)

done to get spark:


In [5]:
data = spark.read.csv('data.csv', inferSchema=True, header=True)

In [6]:
data.columns

['CUSTOMER_ID',
 'COB',
 'FULL_NAME',
 'GENDER',
 'MARITAL_STATUS',
 'AGE',
 'MB_TENURE',
 'SECTOR',
 'EKYC_LEVEL',
 'CHANNEL',
 'ASSET_CLASS',
 'NBR_TXN_DAY',
 'NO_TXN_TM',
 'NBR_TXN_DAY_T_2',
 'NBR_TXN_DAY_T_3',
 'NBR_TXN_DAY_6M',
 'NBR_CREDIT_TXN_T_1',
 'NBR_DEBIT_TXN_T_1',
 'AMT_TOT_CREDIT_TXN_T_1',
 'AMT_TOT_DEBIT_TXN_T_1',
 'NBR_CREDIT_TXN_T_2',
 'NBR_DEBIT_TXN_T_2',
 'AMT_TOT_CREDIT_TXN_T_2',
 'AMT_TOT_DEBIT_TXN_T_2',
 'NBR_CREDIT_TXN_T_3',
 'NBR_DEBIT_TXN_T_3',
 'AMT_TOT_CREDIT_TXN_T_3',
 'AMT_TOT_DEBIT_TXN_T_3',
 'TOTAL_TXN_AMT_T_1',
 'TOTAL_NBR_TXN_T_1',
 'TOTAL_TXN_AMT_T_2',
 'TOTAL_NBR_TXN_T_2',
 'TOTAL_TXN_AMT_T_3',
 'TOTAL_NBR_TXN_T_3',
 'FIRST_DATE_OPEN_credit',
 'CONTRACT_COUNTS_credit',
 'INACTIVE_DAYS_credit',
 'TOTAL_TXN_NBR_T_1_credit',
 'TOTAL_TXN_NBR_T_2_credit',
 'TOTAL_TXN_NBR_T_3_credit',
 'TOTAL_TXN_NBR_6M_credit',
 'TOTAL_TXN_AMT_T_1_credit',
 'TOTAL_TXN_AMT_T_2_credit',
 'TOTAL_TXN_AMT_T_3_credit',
 'TOTAL_TXN_AMT_6M_credit',
 'FIRST_DATE_OPEN_DB',
 'CONTRAC

In [7]:
data.count()

563337

In [8]:
data.select('pd_name', 'pd_code').show()

+--------------------+--------+
|             pd_name| pd_code|
+--------------------+--------+
|001-Our VISA Cust...| MCCRDEB|
|001-VISA EMV Clas...|  MBVSCP|
|001-Diamond Local...|DIAMONDM|
|001-VISA EMV Clas...|  MBVSCP|
|001-VISA Gold Car...|  MBVSGP|
|001-UPI Debit Vir...|UPIVTCRD|
|001-UPI Debit Vir...|UPIVTCRD|
|001-VS Debit Clas...|  VSDBCM|
|001-VS Debit Clas...|  VSDBCM|
|001-Our VISA Cust...| MCCRDEB|
|001-Our VISA Cust...| MCCRDEB|
|001-VISA Platinum...|MBVSPPPR|
|001-VISA Platinum...|MBVSPPPR|
|001-Diamond Local...|DIAMONDM|
|001-BankPlus Prep...|   BPP02|
|001-VS Debit Clas...|  VSDBCM|
|001-VISA EMV Clas...|  MBVSCP|
|001-MB JCB EMV Cr...|MBJCBEPP|
|001-Diamond Local...|DIAMONDM|
|001-VISA Platinum...|MBVSPPPP|
+--------------------+--------+
only showing top 20 rows



In [9]:
column_data_types = data.dtypes
columns = [data_type for column_name, data_type in column_data_types ]
# non_numeric_columns = [column_name for column_name, data_type in column_data_types if data_type != 'int' and data_type != 'double'] 
for i in range(len(columns)):
    if columns[i] == 'bigint' or columns[i] == 'string' or columns[i] == 'vector':
        print(column_data_types[i])

('FULL_NAME', 'string')
('GENDER', 'string')
('MARITAL_STATUS', 'string')
('SECTOR', 'string')
('EKYC_LEVEL', 'string')
('CHANNEL', 'string')
('OVERDRAFT_LOAN_TENURE', 'string')
('pd_code', 'string')
('pd_name', 'string')
('acc_no', 'string')


In [10]:
LABEL_COL = 'pd_code'
LABEL_OUTPUT = LABEL_COL + '_'
FEAT = [
 'NBR_TXN_DAY',
 'NBR_TXN_DAY_T_2',
 'AMT_TOT_CREDIT_TXN_T_2'
]
data = data.fillna(value={**{x: 0.0 for x in FEAT}})
# data = data.sample(0.01)



In [11]:
data.show()

+-----------+----------+--------------------+------+--------------+---+------------+------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------

In [12]:
labelIndexer = StringIndexer(inputCol=LABEL_COL, outputCol=LABEL_OUTPUT)
indexer_df = labelIndexer.fit(data)
data = indexer_df.transform(data)

assembler = VectorAssembler(inputCols=FEAT, outputCol="features")
data = assembler.transform(data)

In [13]:
train, test = data.randomSplit([0.8, 0.2], seed=10)


In [14]:
train.show()

+-----------+----------+--------------------+------+--------------+---+------------+------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------

In [15]:
int(test.select(LABEL_COL).distinct().count())

127

In [19]:
params_setting = {'max_depth': 5,
                  'learning_rate': 0.03,
                  'n_estimators': 10,
                  'colsample_bylevel': 0.5,
                  'colsample_bynode': 0.5,
                  'colsample_bytree': 0.5,
                  'num_classes': int(data.select(LABEL_COL).distinct().count()),
                  'num_workers' : 1}


In [ ]:
from xgboost.spark import SparkXGBClassifier
xgb = SparkXGBClassifier(label_col=LABEL_OUTPUT, features_col="features", **params_setting)
# stages = [labelIndexer,assembler, xgb]
# pipeline = Pipeline(stages=stages)
ModelPipeline = xgb.fit(train)
predTest = ModelPipeline.transform(test)

In [ ]:
data.select(FEAT)

In [ ]:
@udf
def orderByColValue(colValues):
    return [x[1] for x in sorted(zip(colValues, SORTED_LIST), key=lambda x: x[0], reverse=True)][:3]
SORTED_LIST = ModelPipeline.stages[0].labels
predTest = predTest.withColumn("product", orderByColValue('probability'))

In [ ]:
predTest.select(FEAT+['probability','product']).show(truncate=False)